# CONSUMINDO O LIVRO DE REGRAS

In [ ]:
# Instalando a biblioteca LlamaIndex
!pip install -q llama-index

In [ ]:
# Importando a classe que permite carregar documentos de um diretório
from llama_index.core import SimpleDirectoryReader

In [ ]:
# Criando uma instância do SimpleDirectoryReader para carregar documentos do diretório 'documentos'
regras = SimpleDirectoryReader(input_dir='Documentos')

In [ ]:
regras.input_files

In [ ]:
doc_regras = regras.load_data()

In [ ]:
full_text = "\n".join([doc.text for doc in doc_regras])

In [ ]:
import re

# Ajuste a expressão se sua estrutura for diferente
pattern = re.compile(r"(R\d{2})\s*[-–]?\s*(.*?)(?=(R\d{2})|$)", re.DOTALL)
matches = pattern.findall(full_text)

# Cada match vira um dicionário com id e texto
regras_divididas = []
for match in matches:
    rule_id, rule_text, _ = match
    regras_divididas.append({
        "id": rule_id.strip(),
        "text": rule_text.strip()
    })

In [ ]:
from llama_index.core import Document

docs_regras_formatadas = [
    Document(
        text=regra['text'],
        metadata={"rule_id": regra['id']}
    )
    for regra in regras_divididas
]

In [ ]:
regras_divididas

In [ ]:
regras_divididas[0]['id']

In [ ]:
regras_divididas[0]['text']

# IMPORTANDO O EMBEDDING

In [ ]:
# Instalando o pacote para utilizar modelos de embedding da Hugging Face com o LlamaIndex
!pip install -q llama-index-embeddings-huggingface

In [ ]:
# Importando a classe HuggingFaceEmbedding para utilizar modelos de embedding da Hugging Face no LlamaIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
# Classe personalizada para adaptar a assinatura esperada pelo Chroma
class ChromaEmbeddingWrapper:
    def __init__(self, model_name): # Inicializa o modelo de embeddings do Hugging Face com o nome especificado
        self.model = HuggingFaceEmbedding(model_name=model_name)

    def __call__(self, input): # Converte a entrada para um formato compatível com o HuggingFaceEmbedding
        return self.model.embed(input)

In [ ]:
# Definindo o modelo de embedding usado pelo chroma
embed_model_chroma = ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

# SETUP INICIAL DO BANCO VETORIAL (CHROMA DB)

In [ ]:
# Instalando o pacote para integrar o Chroma como um armazenamento de vetores no LlamaIndex
!pip install -q llama-index-vector-stores-chroma

In [ ]:
# Importando o ChromaDB
import chromadb

# Criando um cliente persistente do ChromaDB, armazenando os dados no diretório './chroma_db'
db = chromadb.PersistentClient(path='db_regras')

In [ ]:
# Atribuindo o cliente a uma variável para uso posterior
chroma_client = db

# Tenta obter uma coleção existente ou criar uma nova, caso não exista
try:
  chroma_collection = chroma_client.get_or_create_collection('regras_index')

# Captura e exibe qualquer erro que ocorra durante a criação ou carregamento da coleção
except Exception as e:
  print(f'Erro ao carregar ou criar coleção: {e}')

In [ ]:
# Importando a classe para integrar o Chroma como um armazenamento de vetores no LlamaIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

# Importando a classe que gerencia o armazenamento de índices e vetores
from llama_index.core import StorageContext

In [ ]:
# Criando uma instância do ChromaVectorStore para armazenamento de vetores
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Configurando um contexto de armazenamento com o vector_store como armazenamento de vetores
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# CRIANDO UM INDICE DE ARMAZENAMENTO DE VETORES

In [ ]:
# Definindo o modelo de embedding
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

In [ ]:
# Importando a classe VectorStoreIndex do LlamaIndex, que permite criar um índice de armazenamento de vetores
from llama_index.core import VectorStoreIndex

# Criando um índice de armazenamento de vetores
index_regras = VectorStoreIndex(docs_regras_formatadas, storage_context=storage_context, embed_model=embed_model)

In [ ]:
# Importando a função do LlamaIndex, que permite carregar um índice previamente salvo
from llama_index.core import load_index_from_storage

# Carregando um índice existente a partir do contexto de armazenamento
index_regras = load_index_from_storage(storage_context, embed_model=embed_model)

# DEFININDO O MODELO LLM A SER UTILIZADO

In [ ]:
# Instalando o pacote `llama-index-llms-groq` para integrar o Groq com o LlamaIndex
!pip install -q llama-index-llms-groq

In [ ]:
# Recuperando a chave da API da Groq armazenada nos segredos do usuário no Google Colab
from google.colab import userdata
GROQ_API = userdata.get('GROQ_API')

In [ ]:
# Importando a classe Groq para utilizar o modelo de linguagem da Groq
from llama_index.llms.groq import Groq

# Criando uma instância do Groq com o modelo 'llama3-70b-8192' e a chave de API fornecida
llm = Groq(model='llama3-70b-8192', api_key=GROQ_API)

In [ ]:
# Configura o `query_engine` a partir do index
query_engine = index_regras.as_query_engine(llm=llm, similarity_top_k=10)

In [ ]:
query_engine.query('Quais são as regras contratuais?').response

# INPUT DO ARQUIVO DO USUARIO

In [ ]:
contrato = SimpleDirectoryReader(input_dir="Documentos_User").load_data()
texto_contrato = "\n".join([doc.text for doc in contrato])

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=500, chunk_overlap=250)
contrato_chunks = parser.split_text(texto_contrato)

In [ ]:
len(contrato_chunks)

# PASSA CHUNK POR CHUNK E VERIFICA QUAIS REGRAS SE APLICAM A ELES

# PROMPT

In [ ]:
for chunk in enumerate(contrato_chunks):

        prompt = f"""
        Você atuará como especialista jurídico, com a finalidade de identificar eventuais desconformidades em contratos de prestação de serviços.

          Para tanto, deverá utilizar como base interpretativa o conjunto de regras contratuais constantes na base de dados.

          # Objetivo:
          Analisar o contrato com base nas regras fornecidas, indicando:

          ## 1. Regras Faltantes
          Liste as regras da base que não estão presentes no contrato.

          Para cada regra faltante, apresente:
          - **Título da Regra (código e nome)**
          - **Justificativa:** Explique por que a regra não se aplica ou não foi contemplada.
          - **Avaliação:** Sugira uma cláusula para ser inserida no contrato, a fim de adequá-lo à regra.

          ## 2. Regras Presentes no Documento
          Para cada regra presente no contrato:

          1. Informe o código e nome da regra.
          2. Verifique se há **desconformidade**:
            - Se **não houver**, diga "Desconformidade: Não há" e descreva a cláusula correspondente que comprova a conformidade.
            - Se **houver**, apresente:
              - **Desconformidade:** Trecho literal do contrato que viola ou não atende plenamente à regra.
              - **Regra Violada:** Transcreva o conteúdo da regra.
              - **Avaliação:** Sugira como o trecho pode ser ajustado para estar em conformidade.

          # Formato de resposta esperado:

          ### 1. Regras Faltantes

          #### RXX – Nome da Regra
          - **Justificativa:** [texto explicando ausência]
          - **Avaliação:**
            > Cláusula sugerida para conformidade: "[cláusula sugerida]"

          ---

          ### 2. Regras Presentes no Documento

          #### RXX – Nome da Regra
          - **Desconformidade:** [Sim / Não]
          - **Trecho do Contrato:** [se houver desconformidade]
          - **Regra Violada:** [texto da regra, se aplicável]
          - **Avaliação:**
            > [sugestão de cláusula ou ajuste]

          ---

          # Regras constantes na base de dados:
          {regras_divididas}

          # Conteúdo do contrato de prestação de serviços:
          {chunk}

          Responda em PT-BR com linguagem técnica e jurídica.
              """

resposta = query_engine.query(prompt)

In [ ]:
texto = resposta.response

In [ ]:
texto

# Transforma em PDF

In [ ]:
pip install fpdf

In [ ]:
from fpdf import FPDF
import unicodedata

In [ ]:
def remover_unicode(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

In [ ]:
texto_limpo = remover_unicode(texto)


In [ ]:
def gerar_pdf(texto, nome_arquivo="relatorio_contrato.pdf"):

    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    # Divide o texto em linhas para caber no PDF
    for linha in texto.split('\n'):
        pdf.multi_cell(0, 10, linha)

    pdf.output(nome_arquivo)

gerar_pdf(texto_limpo)

# Tentativas invalidadas

In [ ]:
from collections import defaultdict

# Dicionário que relaciona chunks aos IDs das regras que se aplicam
chunk_regras_aplicadas = defaultdict(list)

In [ ]:


relatorio_avaliacao = []

for i, chunk in enumerate(contrato_chunks):
    for regra in regras_divididas:
        regra_id = regra["id"]
        regra_texto = regra["text"]

        prompt = f"""
Com base na seguinte regra:

{regra_texto}

Responda com SIM ou NÃO: essa regra se aplica ao seguinte trecho do contrato?

Trecho do contrato:
{chunk}
"""

        resposta = query_engine.query(prompt)

        if "sim" in resposta.response.lower():
            chunk_regras_aplicadas[i].append(regra_id)


In [ ]:
for chunk_id in sorted(chunk_regras_aplicadas):
    regras_aplicadas = ", ".join(chunk_regras_aplicadas[chunk_id])
    print(f"Chunk {chunk_id}: {regras_aplicadas}")